<a href="https://www.kaggle.com/code/aayushsin7a/toronto-crime-classification?scriptVersionId=171312752" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Import libraries 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
#Modeling:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#Testing:

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print("Libraries loded successfully")

/kaggle/input/toronto-crime-major-crime/Major_Crime_Indicators_Open_Data.csv
Libraries loded successfully


In [2]:
# Read the dataset 
df = pd.read_csv("/kaggle/input/toronto-crime-major-crime/Major_Crime_Indicators_Open_Data.csv")

In [3]:
df.head(5)

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-8.841785e+06,5.410286e+06,1,GO-20141260127,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1420,110,Assault Bodily Harm,Assault,84,Little Portugal,84,Little Portugal (84),-79.427105,43.642517
1,-8.838126e+06,5.412178e+06,2,GO-20141263725,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1420,100,Assault With Weapon,Assault,78,Kensington-Chinatown,78,Kensington-Chinatown (78),-79.394234,43.654811
2,-8.841053e+06,5.409639e+06,3,GO-20141264272,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1420,100,Assault With Weapon,Assault,85,South Parkdale,85,South Parkdale (85),-79.420532,43.638306
3,-8.850461e+06,5.420267e+06,4,GO-20141263706,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,113,Weston,113,Weston (113),-79.505043,43.707365
4,-8.840470e+06,5.431266e+06,5,GO-20141263710,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,151,Yonge-Doris,51,Willowdale East (51),-79.415293,43.778743


In [4]:
df.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [5]:
df.shape

(384687, 31)

In [6]:
df.rename(columns={'OCC_YEAR':'occurrenceyear','OCC_MONTH':'occurrencemonth','OCC_DAY':'occurrenceday',
                   'OCC_DOY':'occurrencedayofyear','OCC_DOW':'occurrencedayofweek','OCC_HOUR':'occurrencehour',
                   'HOOD_158':'Hood_ID','MCI_CATEGORY':'MCI','PREMISES_TYPE':'premisetype','DIVISION':'Division'},inplace=True)

In [7]:
#list of relevant columns for model
col_list = ['occurrenceyear',	'occurrencemonth','occurrenceday','occurrencedayofyear','occurrencedayofweek','occurrencehour','MCI',	'Division',	'Hood_ID','premisetype']

#dataframe created from list of relevant columns

df2 = df[col_list]
df2 = df2[df2['occurrenceyear'] > 2013] #drop "stale" crimes, where occurence is before 2014. Since data set is filtered based on reported date, we're ignoring these old crimes.

In [8]:
df2.head()

,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,premisetype
0,2014.0,January,1.0,1.0,Wednesday,1,Assault,D14,84,Commercial
1,2014.0,January,1.0,1.0,Wednesday,18,Assault,D52,78,Apartment
2,2014.0,January,1.0,1.0,Wednesday,21,Assault,D14,85,Apartment
3,2014.0,January,1.0,1.0,Wednesday,18,Assault,D12,113,House
4,2014.0,January,1.0,1.0,Wednesday,18,Assault,D32,151,Apartment


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 383121 entries, 0 to 384686
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   occurrenceyear       383121 non-null  float64
 1   occurrencemonth      383121 non-null  object 
 2   occurrenceday        383121 non-null  float64
 3   occurrencedayofyear  383121 non-null  float64
 4   occurrencedayofweek  383121 non-null  object 
 5   occurrencehour       383121 non-null  int64  
 6   MCI                  383121 non-null  object 
 7   Division             383121 non-null  object 
 8   Hood_ID              383121 non-null  object 
 9   premisetype          383121 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 32.2+ MB


# Factorize dependent variable column:

In [10]:
'''
Signature:
pd.factorize(
    values,
    sort: 'bool' = False,
    use_na_sentinel: 'bool' = True,
    size_hint: 'int | None' = None,
) -> 'tuple[np.ndarray, np.ndarray | Index]'
Docstring:
Encode the object as an enumerated type or categorical variable.

This method is useful for obtaining a numeric representation of an
array when all that matters is identifying distinct values. `factorize`
is available as both a top-level function :func:`pandas.factorize`,
and as a method :meth:`Series.factorize` and :meth:`Index.factorize`
'''

"\nSignature:\npd.factorize(\n    values,\n    sort: 'bool' = False,\n    use_na_sentinel: 'bool' = True,\n    size_hint: 'int | None' = None,\n) -> 'tuple[np.ndarray, np.ndarray | Index]'\nDocstring:\nEncode the object as an enumerated type or categorical variable.\n\nThis method is useful for obtaining a numeric representation of an\narray when all that matters is identifying distinct values. `factorize`\nis available as both a top-level function :func:`pandas.factorize`,\nand as a method :meth:`Series.factorize` and :meth:`Index.factorize`\n"

In [11]:
crime_var = pd.factorize(df2['MCI'])#codes the list of crimes to a int64 variable
df2['MCI'] = crime_var[0]
definition_list_MCI = crime_var[1] #create an index reference so we know which crimes are coded to which factors

In [12]:
#factorize independent variables:

#factorize premisetype:

premise_var = pd.factorize(df2['premisetype'])
df2['premisetype'] = premise_var[0]
definition_list_premise = premise_var[1] 

#factorize occurenceyear:

year_var = pd.factorize(df2['occurrenceyear'])
df2['occurrenceyear'] = year_var[0]
definition_list_year = year_var[1] 

#factorize occurencemonth:

month_var = pd.factorize(df2['occurrencemonth'])
df2['occurrencemonth'] = month_var[0]
definition_list_month = month_var[1] 

#factorize occurenceday:

day_var = pd.factorize(df2['occurrenceday'])
df2['occurenceday'] = day_var[0]
definition_list_day = day_var[1] 

#factorize occurencedayofweek:

dayweek_var = pd.factorize(df2['occurrencedayofweek'])
df2['occurrencedayofweek'] = dayweek_var[0]
definition_list_day = dayweek_var[1] 

#factorize division:

division_var = pd.factorize(df2['Division'])
df2['Division'] = division_var[0]
definition_list_division = division_var[1] 

#factorize HOOD_ID:

hood_var = pd.factorize(df2['Hood_ID'])
df2['Hood_ID'] = hood_var[0]
definition_list_hood = hood_var[1] 

#factorize occurencehour:

hour_var = pd.factorize(df2['occurrencehour'])
df2['occurrencehour'] = hour_var[0]
definition_list_hour = hour_var[1] 

#factorize occurencedayofyear:

dayyear_var = pd.factorize(df2['occurrencedayofyear'])
df2['occurrencedayofyear'] = dayyear_var[0]
definition_list_dayyear = dayyear_var[1] 

In [13]:
df2.head()

,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,Division,Hood_ID,premisetype,occurenceday
0,0,0,1.0,0,0,0,0,0,0,0,0
1,0,0,1.0,0,0,1,0,1,1,1,0
2,0,0,1.0,0,0,2,0,0,2,1,0
3,0,0,1.0,0,0,1,0,2,3,2,0
4,0,0,1.0,0,0,1,0,3,4,1,0


In [14]:
#set X and Y:
X = df2.drop(['MCI'],axis=1).values #sets x and converts to an array
print(X)

[[ 0.  0.  1. ...  0.  0.  0.]
 [ 0.  0.  1. ...  1.  1.  0.]
 [ 0.  0.  1. ...  2.  1.  0.]
 ...
 [10.  2. 31. ... 21.  6. 30.]
 [10.  2. 30. ... 21.  2. 29.]
 [10.  2. 30. ... 86.  2. 29.]]


In [15]:
y = df2['MCI'].values

In [16]:
#split the data into train and test sets for numeric encoded dataset:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [17]:
#need to OneHotEncode all the X variables for input into the classification model:
binary_encoder = OneHotEncoder(sparse=False)
encoded_X = binary_encoder.fit_transform(X)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size = 0.30, random_state = 42)

In [19]:
#MODELING AND TESTING:

#Numeric Encoded Model w/ SKLEARN:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) # Predicting the Test set results

print(accuracy_score(y_test, y_pred)) #accuracy at 0.63
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI)) 

0.6461713808434185
[[51825  4006  3632   141  1206]
 [10745  9429  1271   177   146]
 [ 6819  1333  9577    69   465]
 [ 2480   642   476    70    79]
 [ 5428   592   928    33  3368]]
                 precision    recall  f1-score   support

        Assault       0.67      0.85      0.75     60810
Break and Enter       0.59      0.43      0.50     21768
     Auto Theft       0.60      0.52      0.56     18263
     Theft Over       0.14      0.02      0.03      3747
        Robbery       0.64      0.33      0.43     10349

       accuracy                           0.65    114937
      macro avg       0.53      0.43      0.46    114937
   weighted avg       0.62      0.65      0.62    114937



In [20]:
#theft over is pulling down results. Pretty good on Assault (largest sample size) and break and enter 


#One Hot Encoded Model w/ SKLEARN:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train_OH, y_train_OH)
y_pred_OH = classifier.predict(X_test_OH) # Predicting the Test set results

print(accuracy_score(y_test_OH, y_pred_OH)) #modest improvement to 0.648
print(confusion_matrix(y_test_OH, y_pred_OH)) 
print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI)) #modest improvement


0.6592046077416324
[[53553  3212  3215    75   755]
 [11118  9305  1130   141    74]
 [ 7095  1225  9676    43   224]
 [ 2543   642   475    44    43]
 [ 5767   510   868    15  3189]]
                 precision    recall  f1-score   support

        Assault       0.67      0.88      0.76     60810
Break and Enter       0.62      0.43      0.51     21768
     Auto Theft       0.63      0.53      0.58     18263
     Theft Over       0.14      0.01      0.02      3747
        Robbery       0.74      0.31      0.44     10349

       accuracy                           0.66    114937
      macro avg       0.56      0.43      0.46    114937
   weighted avg       0.64      0.66      0.63    114937



In [21]:
#Balanced Class Weight doesn't make a big difference for results:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42, class_weight='balanced')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) 
print(accuracy_score(y_test, y_pred)) #accuracy at 0.63
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI)) 


0.6422126904304097
[[51936  3925  3527   157  1265]
 [11170  8980  1262   183   173]
 [ 7124  1260  9353    76   450]
 [ 2536   617   448    63    83]
 [ 5457   533   845    32  3482]]
                 precision    recall  f1-score   support

        Assault       0.66      0.85      0.75     60810
Break and Enter       0.59      0.41      0.48     21768
     Auto Theft       0.61      0.51      0.56     18263
     Theft Over       0.12      0.02      0.03      3747
        Robbery       0.64      0.34      0.44     10349

       accuracy                           0.64    114937
      macro avg       0.52      0.43      0.45    114937
   weighted avg       0.62      0.64      0.62    114937



In [22]:
#gradientboost performs poorly relative to randomforest

grad_class = GradientBoostingClassifier(learning_rate=0.1,n_estimators = 10, random_state = 42)
grad_class.fit(X_train_OH, y_train_OH)
y_pred_OH = grad_class.predict(X_test_OH) # Predicting the Test set results

print(accuracy_score(y_test_OH, y_pred_OH)) #modest improvement to 0.648
print(confusion_matrix(y_test_OH, y_pred_OH)) 
print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI))

0.5412617346894386
[[60541   111   158     0     0]
 [20999   762     6     1     0]
 [17314    41   908     0     0]
 [ 3676    11    60     0     0]
 [10177    63   109     0     0]]
                 precision    recall  f1-score   support

        Assault       0.54      1.00      0.70     60810
Break and Enter       0.77      0.04      0.07     21768
     Auto Theft       0.73      0.05      0.09     18263
     Theft Over       0.00      0.00      0.00      3747
        Robbery       0.00      0.00      0.00     10349

       accuracy                           0.54    114937
      macro avg       0.41      0.22      0.17    114937
   weighted avg       0.55      0.54      0.40    114937



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
